# Text 4: Word2Vec
**Internet Analytics - Lab 4**

---

**Group:** *H*

**Names:**

* *Antoine Basseto*
* *Andrea Pinto*
* *Jérémy Baffou*


---

#### Instructions

*This is a template for part 4 of the lab. Clearly write your answers, comments and interpretations in Markodown cells. Don't forget that you can add $\LaTeX$ equations in these cells. Feel free to add or remove any cell.*

*Please properly comment your code. Code readability will be considered for grading. To avoid long cells of codes in the notebook, you can also embed long python functions and classes in a separate module. Don’t forget to hand in your module if that is the case. In multiple exercises, you are required to come up with your own method to solve various problems. Be creative and clearly motivate and explain your methods. Creativity and clarity will be considered for grading.*

In [40]:
import pickle
import string
from collections import defaultdict
import re
import numpy as np
from scipy.sparse import csr_matrix
from collections import defaultdict
import json
from utils import *
import gensim
from sklearn.cluster import KMeans

courses = load_json('data/courses.txt')
stopwords = load_pkl('data/stopwords.pkl')

## Redo pre-processing

In [2]:
courses_rdd = sc.parallelize(courses)

In [3]:
courses_processed = courses_rdd.map(lambda c : {"courseId" : c["courseId"], "name" : c["name"], "description" : [word.casefold() for word in c["description"].translate(str.maketrans('', '', string.punctuation)).split() if (word.lower() not in stopwords and not word.isdigit())]})

In [4]:
# remove punctuations, stop words and digits
courses_processed = courses_rdd.map(lambda c :
                                    {"courseId" : c["courseId"], 
                                     "name" : c["name"], 
                                     "description" : [word 
                                                      for word in c["description"]
                                                      .translate(str.maketrans('', '', string.punctuation))
                                                      .split() 
                                                      if (word not in stopwords and not word.isdigit())]})

In [5]:
courses_word_aggregation = courses_processed.flatMap(lambda c : c["description"]) #flatten all words lists

In [6]:
words_number = courses_word_aggregation.count()
words_count = courses_word_aggregation.map(lambda w : (w,1)).reduceByKey(lambda w1,w2 : w1+w2).map(lambda w : (w[1],w[0])).sortByKey(False)
words_freq = words_count.map(lambda w : (w[1],w[0]/words_number))
words_freq_for_plot = np.asarray(words_freq.map(lambda w : w[1]).collect())

In [7]:
really_frequent_indices = np.where(words_freq_for_plot > np.quantile(words_freq_for_plot,0.60)) # maybe change quantile
really_frequent_words = set(words_freq.take(really_frequent_indices[0][-1]))

In [8]:
bag_of_words_per_course = courses_processed.map(lambda c :{"courseId" : c["courseId"], "name" : c["name"], "description" : [w for w in c["description"] if w not in really_frequent_words]}) 

### N-Grams ??

## Get pre-trained model

In [10]:
word_vectors = gensim.models.KeyedVectors.load_word2vec_format('/ix/model.txt')  

## Exercise 4.12 : Clustering word vectors

In [54]:
#default_wv = np.mean(word_vectors.get_normed_vectors(), axis=0)
word_vectors.init_sims(replace=True)
default_wv = word_vectors.get_vector("dog") 
d = defaultdict(lambda: default_wv) 

In [55]:
k = 15
d.update({word: word_vectors.get_vector(word) 
          for word in bag_of_words_per_course.flatMap(lambda c : c["description"]).collect() 
          if word_vectors.has_index_for(word)})
d['a']
#X = d.values()

AttributeError: 'Word2VecKeyedVectors' object has no attribute 'has_index_for'

In [ ]:
kmeans= KMeans(n_clusters=k, random_state=0).fit_predict(X)

## Exercise 4.13 : Document similarity search

## Exercise 4.14: Document similarity search with outside terms